Import Libraries

In [73]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

Configure Environment

In [74]:
# Disabling TensorFlow OneDNN optimizations
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

Initialize MediaPipe Hands

In [75]:
# Initialize MediaPipe Hands module and drawing utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

Data Directory

In [76]:
# Define data directory path
DATA_DIR = './data'

Process Images and Extract Hand Landmarks

In [77]:
# Initialize empty lists for storing data and labels
data = []
labels = []

# Iterate through each subdirectory (representing classes) in the data directory
for dir_ in os.listdir(DATA_DIR):
    # Process each image file in the class directory
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []   # Auxiliary list to store normalized landmark points
        x_ = []         # List to store x-coordinates of landmarks
        y_ = []         # List to store y-coordinates of landmarks

        # Load and convert image to RGB
        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Process image using MediaPipe Hands
        results = hands.process(img_rgb)
        
        # Check if hand landmarks are detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Extract normalized x and y coordinates of each landmark
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    x_.append(x)
                    y_.append(y)

                # Normalize the coordinates by the minimum x and y
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            # Only append to data and labels if data_aux contains 42 elements
            if len(data_aux) == 42:
                data.append(data_aux)
                labels.append(dir_)

Save Processed Data to a Pickle File

In [78]:
# Save data and labels as a pickle file
with open('data.pickle', 'wb') as f:
    pickle.dump({'data': data, 'labels': labels}, f)
print("Data and labels have been saved to 'data.pickle'")

Data and labels have been saved to 'data.pickle'
